# Расчет коэффициентов пролонгации. Техническая реализация

### Шаг 1: Загрузка и предварительная обработка данных

In [1]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import DateOffset

In [2]:
df_prolongations = pd.read_csv('data/prolongations.csv')
df_financial = pd.read_csv('data/financial_data.csv')

#### 1.1. Преобразование финансовой таблицы в 'длинный' формат для удобства анализа временных рядов.

In [3]:
id_vars = ['id', 'Причина дубля', 'Account']

value_vars = [col for col in df_financial.columns if col not in id_vars]

df_financial_long = pd.melt(
    df_financial,
    id_vars=id_vars,
    value_vars=value_vars,
    var_name='shipment_month_str', # Новая колонка для названий месяцев
    value_name='shipment_amount'   # Новая колонка для сумм отгрузок
)

### Шаг 2: Очистка данных и обработка специальных значений

#### 2.1. Исключение досрочно остановленных проектов и обработка текстовых значений.

In [4]:

df_financial_long['shipment_amount'] = df_financial_long['shipment_amount'].replace('в ноль', -1)

df_financial_long['shipment_amount'] = df_financial_long['shipment_amount'].replace(['стоп', 'end'], np.nan)

# Находим уникальные ID всех проектов, у который сумма отгрузок NaN (т.е. мы еще и включаем случаи `стоп`)
stopped_projects = df_financial_long[df_financial_long['shipment_amount'].isna()]['id'].unique()

# Фильтруем датафрейм, удаляя ВСЕ строки, относящиеся к этим ID
df_financial_long = df_financial_long[~df_financial_long['id'].isin(stopped_projects)]

In [5]:
def clean_value(value):
    # Преобразует строковое значение в число (float), очищая от пробелов и заменяя запятую.
    if isinstance(value, str):
        value = value.replace('\xa0', '').replace(' ', '').replace(',', '.')
    try:
        return float(value)
    except (ValueError, TypeError):
        return 0.0

In [6]:
df_financial_long['shipment_amount'] = df_financial_long['shipment_amount'].apply(clean_value)

#### 2.2. Преобразование дат и обработка значений 'в ноль'.

In [7]:
# Преобразуем текстовые даты в формат datetime для обеих таблиц
month_map = {'Январь':'01','Февраль':'02','Март':'03','Апрель':'04','Май':'05','Июнь':'06','Июль':'07','Август':'08','Сентябрь':'09','Октябрь':'10','Ноябрь':'11','Декабрь':'12'}
date_series = df_financial_long['shipment_month_str'].str.split(' ')
df_financial_long['Дата_norm'] = '01-' + date_series.str[0].map(month_map) + '-' + date_series.str[1]
df_financial_long['shipment_month_str'] = pd.to_datetime(df_financial_long['Дата_norm'], format='%d-%m-%Y')

date_series = df_prolongations['month'].str.split(' ')
df_prolongations['Дата_norm'] = '01-' + date_series.str[0].str.capitalize().map(month_map) + '-' + date_series.str[1]
df_prolongations['month'] = pd.to_datetime(df_prolongations['Дата_norm'], format='%d-%m-%Y')

In [8]:
df_prolongations = df_prolongations.drop(columns=['Дата_norm'])
df_financial_long = df_financial_long.drop(columns=['Дата_norm'])

In [9]:
df_financial_long = df_financial_long.sort_values(by=['id', 'shipment_month_str'])

# Заполняем маркеры -1 ('в ноль') предыдущим значением внутри группы каждого проекта
df_financial_long['shipment_amount'] = df_financial_long.groupby('id')['shipment_amount'].transform(
    lambda x: x.replace(-1, method='ffill')
)

# Если маркер -1 остался (например, в самом первом месяце проекта), заменяем его на 0
df_financial_long['shipment_amount'] = df_financial_long['shipment_amount'].replace(-1, 0)

C:\Users\scarl\AppData\Local\Temp\ipykernel_20384\1474639829.py:5: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  lambda x: x.replace(-1, method='ffill')


### Шаг 3: Агрегация и финальная подготовка таблицы

Объединяем данные и суммируем все отгрузки по проекту за месяц, чтобы получить единую, готовую к расчетам таблицу.

In [10]:
df_merged = pd.merge(
    df_prolongations,
    df_financial_long,
    on='id',
    how='inner'
)

In [11]:
df_clean = df_merged.drop(columns=['Account', 'Причина дубля'])

df_agg = df_clean.groupby(['id', 'AM', 'shipment_month_str', 'month'])['shipment_amount'].sum().reset_index()

df_cleanest = df_agg.rename(columns={
    'id': 'project_id',
    'AM': 'account_manager',
    'month': 'project_end_month', # Последний месяц реализации проекта
    'shipment_month_str': 'shipment_month', # Месяц, за который произведена отгрузка
    'shipment_amount': 'shipment_sum' # Сумма отгрузки
})

In [12]:
df_cleanest.head()

,project_id,account_manager,shipment_month,project_end_month,shipment_sum
0,190,Васильев Артем Александрович,2022-11-01,2023-02-01,149080.0
1,190,Васильев Артем Александрович,2022-11-01,2023-12-01,149080.0
2,190,Васильев Артем Александрович,2022-12-01,2023-02-01,149100.0
3,190,Васильев Артем Александрович,2022-12-01,2023-12-01,149100.0
4,190,Васильев Артем Александрович,2023-01-01,2023-02-01,177595.0


### Шаг 4: Расчет коэффициентов

Реализация основного цикла для итеративного расчета К1 и К2.

In [13]:
managers = df_cleanest['account_manager'].unique().tolist()
all_managers_label = 'Весь отдел'
managers_and_total = managers + [all_managers_label]

report_months = pd.to_datetime([f'2023-{i}-01' for i in range(1, 13)])

results = []

Сам цикл:

In [14]:
for manager in managers_and_total:
    
    if manager == all_managers_label:
        manager_df = df_cleanest
    else:
        manager_df = df_cleanest[df_cleanest['account_manager'] == manager]

    total_k1_numerator, total_k1_denominator = 0, 0
    total_k2_numerator, total_k2_denominator = 0, 0

    for current_month in report_months:

        # Расчет Коэффициента пролонгации на 1-й месяц
        prev_month = current_month - DateOffset(months=1)

        # Знаменатель: Находим проекты, завершившиеся в прошлом месяце, и сумму их отгрузки
        # за тот же месяц
        k1_base_df = manager_df[
            (manager_df['project_end_month'] == prev_month) &
            (manager_df['shipment_month'] == prev_month)
        ]
        k1_denominator = k1_base_df['shipment_sum'].sum()

        # Числитель: Для этих же проектов смотрим их отгрузку в текущем месяце
        k1_prolonged_project_ids = k1_base_df['project_id'].unique()
        k1_numerator = manager_df[
            (manager_df['project_id'].isin(k1_prolonged_project_ids)) &
            (manager_df['shipment_month'] == current_month)
        ]['shipment_sum'].sum()

        # Расчет Коэффициента пролонгации на 2-й месяц
        month_minus_2 = current_month - DateOffset(months=2)

        # Знаменатель:
        # 1. Находим ID проектов, завершившихся 2 месяца назад
        ended_m2_ids = manager_df[manager_df['project_end_month'] == month_minus_2]['project_id'].unique()

        # 2. Находим ID проектов из 1-ого пункта, у которых была отгрузка в прошлом месяце (т.е. они были пролонгированы на 1-й месяц)
        prolonged_in_1st_month_ids = manager_df[
            (manager_df['project_id'].isin(ended_m2_ids)) &
            (manager_df['shipment_month'] == prev_month) &
            (manager_df['shipment_sum'] > 0)
        ]['project_id'].unique()

        # 3. Находим ID проектов для нашей базы - те, что завершились 2 месяца назад и не были пролонгированы на 1-й месяц
        k2_base_project_ids = np.setdiff1d(ended_m2_ids, prolonged_in_1st_month_ids)

        # 4. Считаем сумму отгрузки для этих непролонгированных проектов за их последний месяц (т.е. 2 месяца назад)
        k2_denominator = manager_df[
            (manager_df['project_id'].isin(k2_base_project_ids)) &
            (manager_df['shipment_month'] == month_minus_2)
        ]['shipment_sum'].sum()

        # Числитель: Для этих же непролонгированных проектов ищем отгрузку в текущем месяце
        k2_numerator = manager_df[
            (manager_df['project_id'].isin(k2_base_project_ids)) &
            (manager_df['shipment_month'] == current_month)
        ]['shipment_sum'].sum()

        # Рассчитываем коэффициенты, избегая деления на ноль
        k1 = k1_numerator / k1_denominator if k1_denominator > 0 else 0
        k2 = k2_numerator / k2_denominator if k2_denominator > 0 else 0

        # Сохраняем результат за месяц в наш список
        results.append({
            'Менеджер': manager,
            'Метрика': 'Коэффициент 1',
            'Месяц': current_month,
            'Значение': k1
        })
        results.append({
            'Менеджер': manager,
            'Метрика': 'Коэффициент 2',
            'Месяц': current_month,
            'Значение': k2
        })

        # Добавляем месячные суммы к годовым
        total_k1_numerator += k1_numerator
        total_k1_denominator += k1_denominator
        total_k2_numerator += k2_numerator
        total_k2_denominator += k2_denominator
    
    # Рассчитываем годовые коэффициенты
    annual_k1 = total_k1_numerator / total_k1_denominator if total_k1_denominator > 0 else 0
    annual_k2 = total_k2_numerator / total_k2_denominator if total_k2_denominator > 0 else 0

    # Сохраняем годовой результат
    results.append({
        'Менеджер': manager,
        'Метрика': 'Коэффициент 1 (Год)',
        'Месяц': 'Годовой',
        'Значение': annual_k1
    })
    results.append({
        'Менеджер': manager,
        'Метрика': 'Коэффициент 2 (Год)',
        'Месяц': 'Годовой',
        'Значение': annual_k2
    })

### Шаг 5: Формирование и экспорт отчета в Excel

Сводим полученные результаты в `pivot`-таблицу и выгружаем в Excel-файл, применяя базовое форматирование.

In [15]:
results_df = pd.DataFrame(results)

report_pivot = results_df.pivot_table(
    index='Менеджер', 
    columns=['Месяц', 'Метрика'], 
    values='Значение'
)

file_name = 'analytical_report.xlsx'
with pd.ExcelWriter(file_name, engine='xlsxwriter') as writer:
    report_pivot.to_excel(writer, sheet_name='Отчет по пролонгациям', merge_cells=True)
    
    workbook = writer.book
    worksheet = writer.sheets['Отчет по пролонгациям']
    
    worksheet.set_column('A:A', 30)
    worksheet.set_column('B:AZ', 15)